In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [4]:
# load dataset into pandas dataframe
diplomacy = pd.read_json('data/diplomacy_data.json')
diplomacy = diplomacy.set_index('idx')
diplomacy.head()

,seasons,game,betrayal,people
idx,,,,
0,"[{'season': 1906.5, 'interaction': {'victim': ...",74,True,AT
1,"[{'season': 1911.5, 'interaction': {'victim': ...",165,False,EG
2,"[{'season': 1911.0, 'interaction': {'victim': ...",157,False,AR
3,"[{'season': 1902.0, 'interaction': {'victim': ...",58,False,AR
4,"[{'season': 1910.0, 'interaction': {'victim': ...",45,False,IT
